In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
import sys
import h5py
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

np.random.seed(0)

### Data utils

In [0]:
def get_dataset_name(file_name_with_dir):
    filename_without_dir = file_name_with_dir.split('/')[-1]
    temp = filename_without_dir.split('_')[:-1]
    dataset_name = "_".join(temp)
    return dataset_name


def normalize(matrix, class_id, maxs_array, mins_array):
  maxs = maxs_array[:, class_id]
  maxs = maxs.reshape(maxs.shape[0],1)
  mins = mins_array[:, class_id]
  mins = mins.reshape(mins.shape[0],1)
  return (matrix-mins)/(maxs-mins)

def subsample(matrix, downsampling_factor=10):
  return matrix[:, ::downsampling_factor]

### Data generators

In [0]:
def train_val_generator(train_data_dir, mode='training', downsampling_factor=10, val_split=0.2, shuffle=False, window_size=1000, window_shift=50, maxs_array=None, mins_array=None):
  '''
  Generate 4 chunk, one one class
  '''
  
  class_names=['rest', 'task_motor', 'task_story_math', 'task_working_memory']
  
  subject_id_list = [113922, 164636]
  sub_idx = 0
  chunk_id = 1

  while True:
      
    np.random.seed(chunk_id)
    
    # Initialize the dataset
    data_X = []
    data_Y = []

    subject_id = subject_id_list[sub_idx]

    for class_id, class_name in enumerate(class_names):
      filename_path = train_data_dir + '/' + class_name + '_' + str(subject_id) + '_' + str(chunk_id) + '.h5'

      ### Supplied code
      with h5py.File(filename_path,'r') as f:
        dataset_name = get_dataset_name(filename_path)
        matrix = f.get(dataset_name)[()]
      ###

      matrix = normalize(matrix, class_id, maxs_array, mins_array)
      matrix = subsample(matrix, downsampling_factor)
      matrix = matrix.T
      for i in range(window_size-1, matrix.shape[0], window_shift):
        x_instance = matrix[i-window_size+1: i+1, :]
        data_X.append(x_instance)
        data_Y.append(class_id)


    data_X = np.asarray(data_X)
    data_X = data_X.astype(np.float16)
    data_Y = np.asarray(data_Y).astype(np.int8)


    # Shuffle
    if shuffle:
      indices = np.arange(data_X.shape[0])
      np.random.shuffle(indices)
      data_X = data_X[indices,:,:]
      data_Y = data_Y[indices]
        
    n_instances = data_X.shape[0]
    n_instances_train = round((1-val_split)*n_instances)
    
    if mode == 'training':
      # Take first 'n_instances_train' instances of the dataset
      print("\nYielding: Chunk {} (training)".format(chunk_id))
      yield data_X[:n_instances_train,:,:], data_Y[:n_instances_train]


    if mode == 'validation':
      # Take the remaining instances as validation set
      yield data_X[n_instances_train:,:,:], data_Y[n_instances_train:]


    chunk_id += 1
    if chunk_id > 8:
      chunk_id = 1

      sub_idx += 1
      if sub_idx > 1:
        sub_idx = 0

      #raise Exception("All files yielded")
  

def test_generator(test_data_dir, subject_id, downsampling_factor=10, window_size=1000, window_shift=50, maxs_array=None, mins_array=None):
  '''
  Generate 4 chunk, one one class
  '''
  
  class_names=['rest','task_motor','task_story_math','task_working_memory']

  file_names = sorted(os.listdir(test_data_dir))

  while True:

    for file_name in file_names:
      # Initialize the dataset
      test_data_X = []
      test_data_Y = []
      
      filename_path = test_data_dir + file_name
      class_name = file_name.split('_')[:-2]
      class_id = class_names.index(class_name)

      ### Supplied code
      with h5py.File(filename_path,'r') as f:
        dataset_name = get_dataset_name(filename_path)
        matrix = f.get(dataset_name)[()]
      ###

      matrix = normalize(matrix, class_id, maxs_array, mins_array)
      matrix = subsample(matrix, downsampling_factor)
      matrix = matrix.T
      for i in range(window_size-1, matrix.shape[0], window_shift):
        x_instance = matrix[i-window_size+1: i+1, :]
        test_data_X.append(x_instance)
        test_data_Y.append(class_id)

      test_data_X = np.asarray(test_data_X).astype(np.float16)
      test_data_Y = np.asarray(test_data_Y).astype(np.int8)

      # print("Yielding chunk set: ",chunk_id)
      yield test_data_X, test_data_Y



### Configuration


In [2]:
# CROSS train 

train_data_dir = "/content/drive/My Drive/DL-final_project/Data/Cross/train/"
cross_base_dir = "/content/drive/My Drive/DL-final_project/Data/Cross"

train_subject_id_list = [113922, 164636]

class_names = ['rest', 'task_motor', 'task_story_math', 'task_working_memory']
n_sensors = 248        # FIXED: No. of features (sensors) in the signal
steps_per_file = 35624 # FIXED


# Data preprocessing params
DOWNSAMPLING_FACTOR = 10
WINDOW_SIZE = 100
WINDOW_SHIFT = 10
VAL_SPLIT = 0.2
SHUFFLE = True

# Baseline hyper-params
HYPER_PARAMS = {'lr': 0.001, 'batch_norm':False, 'dropout_prob':0.0}
N_CONV_LAYERS = 3
KERNELS = [32,32,64]
KERNEL_SIZES = [3,3,3]

# Training
EPOCHS = 50

# Normalization stuff
maxs_file_path = "/content/drive/My Drive/DL-final_project/data_stats/cross_train_maxs.txt"
mins_file_path = "/content/drive/My Drive/DL-final_project/data_stats/cross_train_mins.txt"
maxs_array = np.loadtxt(maxs_file_path, delimiter=',')
mins_array = np.loadtxt(mins_file_path, delimiter=',')

OSError: ignored

### Model definition

In [0]:
def CNN_model_cross(window_size, n_conv_layers, kernels, kernel_sizes, hyper_params={'lr':0.001, 'batch_norm':False, 'dropout_prob':0.0}, initializer='glorot_uniform'):
    tf.random.set_seed(0)
    tf.keras.backend.clear_session()

    model = models.Sequential()
    model.add(layers.Conv1D(filters=kernels[0], kernel_size=kernel_sizes[0], strides=1, 
                            activation='relu', 
                            kernel_initializer=initializer,
                            input_shape=(window_size, 248)))

    # model.add(layers.MaxPooling1D(pool_size=2))

    if hyper_params['batch_norm']:
      model.add(layers.BatchNormalization())

    for i in range(1, n_conv_layers):
        model.add(layers.Conv1D(filters=kernels[i], kernel_size=kernel_sizes[i], strides=1, activation='relu', kernel_initializer=initializer))
        model.add(layers.MaxPooling1D(pool_size=2))
        if hyper_params['batch_norm']:
          model.add(layers.BatchNormalization())
    
    model.add(layers.Flatten())

    model.add(layers.Dense(units=64, activation='relu', kernel_initializer=initializer))
    model.add(layers.Dropout(hyper_params['dropout_prob']))

    model.add(layers.Dense(units=64, activation='relu', kernel_initializer=initializer))
    model.add(layers.Dropout(hyper_params['dropout_prob']))

    model.add(layers.Dense(units=4, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hyper_params['lr']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    return model


## Experiment 1: Number of Layers

In [0]:
# Sweep config

n_conv_layers_list = [3, 4, 5, 6]
kernels_list = [[32,32,64],
                [32,32,64,64],
                [32,32,64,64,128],
                [32,32,64,64,128,256]]
kernel_sizes_list = [[3,3,3],
                     [3,3,3,3],
                     [3,3,3,3,3],
                     [3,3,3,3,3,3]]


# Callback config
base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt1-nLayers/logs/"

In [0]:
for i, n_conv_layers in enumerate(n_conv_layers_list):
  # Set up callback
  log_dir = base_log_dir + str(n_conv_layers) + "_layer/"
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  kernels = kernels_list[i]
  kernel_sizes = kernel_sizes_list[i]
  model = CNN_model_cross(WINDOW_SIZE,
                    n_conv_layers, kernels, kernel_sizes, 
                    HYPER_PARAMS)
  
  # Start training
  print("EXPT CASE: N Conv Layers-", n_conv_layers)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])
             

## Experiment 2: Kernel sizes

In [0]:
# Baseline config
N_CONV_LAYERS=3
KERNELS=[32,32,64]

# Sweep config
kernel_sizes_list = [[3,3,3],
                    [5,3,3],
                    [5,5,3],
                    [7,3,3],
                    [7,5,3]]

# Callback config
base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt2-kernelSize/logs/"

In [0]:
for i, kernel_sizes in enumerate(kernel_sizes_list):
  # Set up callback
  log_dir = base_log_dir +  "{}-{}-{}_kernelSize/".format(kernel_sizes[0], kernel_sizes[1], kernel_sizes[2])
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  model = CNN_model_cross(WINDOW_SIZE,
                    N_CONV_LAYERS, KERNELS, kernel_sizes, 
                    HYPER_PARAMS)
  
  # Start training
  print("EXPT CASE:  Kernel sizes-", kernel_sizes)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])
             

## Experiment 3: Batch normalization

In [0]:
# Callback config
base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt3-batchNorm/logs/"

for batch_norm in [False, True]:

  # Set up callback
  log_dir = base_log_dir +  "{}_batchNorm/".format(int(batch_norm))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  model = CNN_model_cross(WINDOW_SIZE,
                    N_CONV_LAYERS, KERNELS, KERNEL_SIZES, 
                    hyper_params={'lr':0.001, 'batch_norm':batch_norm, 'dropout_prob':0.0})
  
  # Start training
  print("EXPT CASE:  Batch norm-", batch_norm)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])
             

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 98, 32)            23840     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 32)            3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 64)            6208      
_________________________________________________________________
flatten (Flatten)            (None, 2944)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               376960    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

## Experiment 4: Dropout regularization

In [0]:
# Sweep config
dropout_prob_list = [0.0, 0.1, 0.15, 0.2]

base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt4-dropout/logs/"

In [0]:
for dropout_prob in dropout_prob_list:
   # Set up callback
  log_dir = base_log_dir +  "{}_dropout/".format(dropout_prob)
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  model = CNN_model_cross(WINDOW_SIZE,
                    N_CONV_LAYERS, KERNELS, KERNEL_SIZES, 
                    hyper_params={'lr':0.001, 'batch_norm':False, 'dropout_prob':dropout_prob})
  
  # Start training
  print("EXPT CASE: Dropout-", dropout_prob)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])

## Experiment 5: Learning rate tuning 

In [0]:
# Sweep config
learning_rate_list = [0.005, 0.007, 0.01, 0.03]

base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt5-learningRate/logs/"

In [0]:
for i, learning_rate in enumerate(learning_rate_list):
  # Set up callback
  log_dir = base_log_dir +  "{}_learningRate/".format(learning_rate)
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  model = CNN_model_cross(WINDOW_SIZE,
                    N_CONV_LAYERS, KERNELS, KERNEL_SIZES, 
                    hyper_params={'lr':learning_rate, 'batch_norm':False, 'dropout_prob':0.0})
  
  # Start training
  print("\nEXPT CASE: Learning rate-", learning_rate)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])

## Experiment 6: Weight initialization

In [0]:
# Sweep config

initializer_list = ['glorot_uniform'
                    'glorot_normal', 
                    'zeros', 
                    'random_normal',
                    'random_uniform']

base_log_dir = "/content/drive/My Drive/DL-final_project/Results/Cross/Expt6-weightInit/logs/"

In [0]:
for initializer in initializer_list:
     # Set up callback
  log_dir = base_log_dir +  "{}_weightInit/".format(initializer)
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False,
      update_freq='epoch', profile_batch=2, embeddings_freq=0,
      embeddings_metadata=None)

  # Initialize data generators
  train_generator = train_val_generator(train_data_dir, subject_id, 
                                                 'training', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                                 WINDOW_SIZE, WINDOW_SHIFT, 
                                                 maxs_array, mins_array)

  val_generator = train_val_generator(train_data_dir, subject_id, 
                                               'validation', DOWNSAMPLING_FACTOR, VAL_SPLIT, SHUFFLE, 
                                               WINDOW_SIZE, WINDOW_SHIFT, 
                                               maxs_array, mins_array)

  # Initialize model
  model = CNN_model_cross(WINDOW_SIZE,
                    N_CONV_LAYERS, KERNELS, KERNEL_SIZES, 
                    HYPER_PARAMS,
                    initializer)
  
  # Start training
  print("\nEXPT CASE: Initializer-", initializer)
  model.fit(train_generator,
              steps_per_epoch = 8,
              validation_data = val_generator,
              validation_steps = 8,
              epochs = EPOCHS,
              callbacks=[tensorboard_callback])